# import libraries

In [1]:
import mysql.connector
import pandas as pd

# Extract

In [2]:
# database information
host = "localhost"
user = "root"
password = ""
database = "bike_store"

# function to extract data
def extract_data(host, user, password, database, table_names):
    # Create a connection
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )

    # Initialize an empty dictionary to store DataFrames
    extracted_data = {}

    # Loop through the table names and extract data
    for table_name in table_names:
        sql_query = "SELECT * FROM {}".format(table_name)
        df = pd.read_sql_query(sql_query, connection)
        
        # Store the DataFrame in the dictionary
        extracted_data[table_name] = df

        # Print the first few rows of each DataFrame
        print("Data from table '{}':".format(table_name))
        print(df.head())
        print("\n")

    # Close the database connection
    connection.close()

    return extracted_data

# Example usage:
host = "localhost"
user = "root"
password = ""
database = "bike_store"
table_names = ["brands", "categories", "customers", "orders", "order_items","products","staffs","stocks","stores"]

extracted_data = extract_data(host, user, password, database, table_names)

Data from table 'brands':
   brand_id   brand_name
0         1      Electra
1         2         Haro
2         3       Heller
3         4  Pure Cycles
4         5      Ritchey


Data from table 'categories':
   category_id        category_name
0            1    Children Bicycles
1            2     Comfort Bicycles
2            3    Cruisers Bicycles
3            4  Cyclocross Bicycles
4            5       Electric Bikes


Data from table 'customers':
   customer_id  first_name last_name           phone                    email  \
0            1       Debra     Burks            None    debra.burks@yahoo.com   
1            2       Kasha      Todd            None     kasha.todd@yahoo.com   
2            3      Tameka    Fisher            None    tameka.fisher@aol.com   
3            4       Daryl    Spence            None     daryl.spence@aol.com   
4            5  Charolette      Rice  (916) 381-6003  charolette.rice@msn.com   

                  street           city state  zip_code  
